In [1]:
!pip install -qU crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sagemaker 2.237.0 requires protobuf<5.0,>=3.12, but you have protobuf 5.29.2 which is incompatible.


In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os
from utils import get_bedrock_client, get_haiku_config, get_sonnet_config
from langchain_community.chat_models import BedrockChat  # Changed this import
from crewai import Agent, Task, Crew

In [4]:
# import os
# from utils import get_openai_api_key

# openai_api_key = get_openai_api_key()
# os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'


bedrock_client = get_bedrock_client()
sonnet_config = get_sonnet_config()  # Using Sonnet config

# Create BedrockChat instance for CrewAI
llm = BedrockChat(
    client=bedrock_client,
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",  # Updated model ID
    streaming=True,
    model_kwargs={
        "temperature": sonnet_config["temperature"],
        "top_p": sonnet_config["top_p"],
        "max_tokens": sonnet_config["max_tokens"],
        "anthropic_version": "bedrock-2023-05-31"
    }
)

# Set model name in environment if needed
os.environ["MODEL_NAME"] = "anthropic.claude-3-sonnet-20240229-v1:0"

## crewAI Tools

In [5]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [19]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    # llm=llm,
    # max_iter=5,
    # memory=True,
)

ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [8]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm=llm,
    max_iter=5,
    memory=True,
)

In [9]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm=llm,
    max_iter=5,
    memory=True,
)

In [10]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm=llm,
    max_iter=5,
    memory=True,
)

## Creating Tasks

In [11]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [12]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [13]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [14]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [15]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=llm,
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [16]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [17]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
Thought: To continuously monitor and analyze market data for AAPL, I will need the help of my Data Analyst co-worker to gather and process the relevant data. I should also consult with the Trading Strategy Developer to identify appropriate statistical models and machine learning techniques for trend analysis and prediction.

Action: Ask question to co-worker
Action Input: {
    "coworker": "Data Analyst",
    "question": "Can you please gather historical stock price data, trading volume data, and any other relevant market data for AAPL over the past 5 years?",
    "context": "We need to continuously monitor and analyze market data for the stock AAPL (Apple Inc.). This data will be used to identify trends and predi

- Display the final result as Markdown.

In [18]:
from IPython.display import Markdown
Markdown(result)

Based on the analysis and recommendations from the team members, here is a comprehensive risk analysis report detailing potential risks and mitigation strategies for the proposed trading strategies for AAPL (Apple Inc.) stock:

I. Long-term Buy-and-Hold Strategy

Potential Risks:
1. Market Risk: The overall stock market can experience significant volatility and downturns, leading to potential losses in the long run.
2. Company-Specific Risk: Apple's performance may decline due to factors such as competition, product failures, or management issues, negatively impacting the stock price.
3. Opportunity Cost Risk: By holding the stock for an extended period, investors may miss out on potential gains from other investment opportunities.

Mitigation Strategies:
- Diversification: Maintain a well-diversified portfolio across different sectors and asset classes to mitigate market risk.
- Continuous Monitoring: Regularly review Apple's financial performance, competitive position, and industry trends to identify potential risks early.
- Rebalancing: Periodically rebalance the portfolio to maintain the desired asset allocation and risk exposure.

II. Active Trading Strategies (Momentum, Breakout, and Swing Trading)

Potential Risks:
1. Whipsaw Risk: The risk of being caught in sudden reversals of price trends, leading to losses.
2. Timing Risk: The risk of entering or exiting a position at the wrong time, missing potential gains or incurring losses.
3. Liquidity Risk: The risk of being unable to execute trades due to low trading volume or market volatility.
4. False Breakout Risk: The risk of entering a position based on a false breakout signal, leading to potential losses.
5. Volatility Risk: The risk of increased volatility during breakout periods, which can lead to wider stop-loss levels and larger potential losses.
6. Overnight Risk: The risk of adverse price movements occurring while positions are held overnight or over weekends.
7. Emotional Risk: The risk of making impulsive or emotional trading decisions, leading to suboptimal entries or exits.
8. Leverage Risk: The risk of using excessive leverage, which can amplify potential losses.

Mitigation Strategies:
- Strict Risk Management: Implement stop-loss orders, position sizing rules, and risk-defined entries and exits to limit potential losses.
- Trend Confirmation: Use multiple technical indicators and chart patterns to confirm the strength and validity of price trends before entering positions.
- Diversification: Diversify across different trading strategies and asset classes to reduce concentration risk.
- Confirmation Signals: Use additional technical indicators or chart patterns to confirm the validity of breakout signals before entering positions.
- Volatility Management: Adjust position sizes and stop-loss levels based on expected volatility during breakout periods.
- Limit Orders: Use limit orders instead of market orders to control execution prices and minimize slippage.
- Discipline and Emotional Control: Develop and adhere to a trading plan, and avoid impulsive or emotional decision-making.
- Proper Position Sizing: Use appropriate position sizing based on account size and risk tolerance to manage leverage risk.

III. Execution Risks

Potential Risks:
1. Market Impact: The act of executing large orders or trades in illiquid stocks can move the market price unfavorably.
2. Slippage: The difference between the expected price of a trade and the price at which the trade is actually executed.
3. Latency: Delays in receiving market data or executing orders can lead to stale prices and missed trading opportunities.
4. Operational Risks: Errors in order entry, fat-finger mistakes, or system failures can lead to unintended trades or losses.
5. Regulatory Risks: Non-compliance with trading regulations can result in penalties or trade rejections.
6. Liquidity Risks: Difficulty in entering or exiting large positions or illiquid stocks without significantly impacting the market price.

Mitigation Strategies:
- Advanced Execution Algorithms: Implement algorithmic trading strategies that slice large orders into smaller child orders and execute them incrementally over time, minimizing market impact.
- Real-time Market Data Feeds: Leverage real-time market data feeds and low-latency infrastructure to minimize latency and adjust order prices dynamically.
- Robust Risk Management Controls: Implement pre-trade risk controls, order validation checks, position limits, and stop-loss orders to mitigate operational risks.
- Regulatory Compliance: Ensure compliance with relevant trading regulations, such as circuit breakers, short-selling restrictions, or position limits.
- Liquidity Monitoring: Monitor liquidity metrics and adjust position sizes or trading strategies accordingly.
- Collaboration with Execution Traders: Work closely with experienced execution traders and brokers to optimize trade execution strategies.

IV. Additional Considerations

- Continuous Monitoring and Adaptation: Regularly review and update trading systems, algorithms, and risk management processes to adapt to changing market conditions and new developments.
- Backtesting and Paper Trading: Conduct thorough backtesting and paper trading before implementing any trading strategy with real capital to validate its effectiveness and identify potential risks.
- Professional Advice: Seek guidance from financial advisors and conduct thorough research to ensure alignment with personal financial goals and risk tolerance.

By implementing these risk mitigation strategies and continuously monitoring and adapting to changing market conditions, traders can effectively manage the potential risks associated with the proposed trading strategies for AAPL stock. It's crucial to maintain a disciplined approach, adhere to risk management principles, and prioritize capital preservation while pursuing potential trading opportunities.